## Import

In [229]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

# pip3 install torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# pip3 install albumentations --user
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# pip3 install torchvision
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [230]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [231]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [232]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [233]:
# 학습용 이미지 폴더 경로 수정
all_img_list = glob.glob(os.path.join(os.path.pardir, os.path.pardir, 'open', 'train', ) + '/*/*')
print('all_img_list :', all_img_list)

all_img_list : ['..\\..\\open\\train\\가구수정\\0.png', '..\\..\\open\\train\\가구수정\\1.png', '..\\..\\open\\train\\가구수정\\10.png', '..\\..\\open\\train\\가구수정\\11.png', '..\\..\\open\\train\\가구수정\\2.png', '..\\..\\open\\train\\가구수정\\3.png', '..\\..\\open\\train\\가구수정\\4.png', '..\\..\\open\\train\\가구수정\\5.png', '..\\..\\open\\train\\가구수정\\6.png', '..\\..\\open\\train\\가구수정\\7.png', '..\\..\\open\\train\\가구수정\\8.png', '..\\..\\open\\train\\가구수정\\9.png', '..\\..\\open\\train\\걸레받이수정\\0.png', '..\\..\\open\\train\\걸레받이수정\\1.png', '..\\..\\open\\train\\걸레받이수정\\10.png', '..\\..\\open\\train\\걸레받이수정\\100.png', '..\\..\\open\\train\\걸레받이수정\\101.png', '..\\..\\open\\train\\걸레받이수정\\102.png', '..\\..\\open\\train\\걸레받이수정\\103.png', '..\\..\\open\\train\\걸레받이수정\\104.png', '..\\..\\open\\train\\걸레받이수정\\105.png', '..\\..\\open\\train\\걸레받이수정\\106.png', '..\\..\\open\\train\\걸레받이수정\\107.png', '..\\..\\open\\train\\걸레받이수정\\108.png', '..\\..\\open\\train\\걸레받이수정\\109.png', '..\\..\\open\\train\\걸레받이수정\\11.pn

In [234]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list

# 학습용 이미지 폴더 경로 수정
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[4])
print('df :', df)

df :                           img_path label
0      ..\..\open\train\가구수정\0.png  가구수정
1      ..\..\open\train\가구수정\1.png  가구수정
2     ..\..\open\train\가구수정\10.png  가구수정
3     ..\..\open\train\가구수정\11.png  가구수정
4      ..\..\open\train\가구수정\2.png  가구수정
...                            ...   ...
3452   ..\..\open\train\훼손\995.png    훼손
3453   ..\..\open\train\훼손\996.png    훼손
3454   ..\..\open\train\훼손\997.png    훼손
3455   ..\..\open\train\훼손\998.png    훼손
3456   ..\..\open\train\훼손\999.png    훼손

[3457 rows x 2 columns]


In [235]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])
print('train :', train)

train :                           img_path label
3322   ..\..\open\train\훼손\878.png    훼손
2089  ..\..\open\train\훼손\1030.png    훼손
2321   ..\..\open\train\훼손\124.png    훼손
2599   ..\..\open\train\훼손\226.png    훼손
2305  ..\..\open\train\훼손\1225.png    훼손
...                            ...   ...
2945   ..\..\open\train\훼손\538.png    훼손
1327   ..\..\open\train\오염\365.png    오염
823    ..\..\open\train\면불량\82.png   면불량
2767   ..\..\open\train\훼손\378.png    훼손
905    ..\..\open\train\몰딩수정\4.png  몰딩수정

[2419 rows x 2 columns]


## Label-Encoding

In [236]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
print('train :', train)

train :                           img_path  label
3322   ..\..\open\train\훼손\878.png     18
2089  ..\..\open\train\훼손\1030.png     18
2321   ..\..\open\train\훼손\124.png     18
2599   ..\..\open\train\훼손\226.png     18
2305  ..\..\open\train\훼손\1225.png     18
...                            ...    ...
2945   ..\..\open\train\훼손\538.png     18
1327   ..\..\open\train\오염\365.png     10
823    ..\..\open\train\면불량\82.png      6
2767   ..\..\open\train\훼손\378.png     18
905    ..\..\open\train\몰딩수정\4.png      7

[2419 rows x 2 columns]


## CustomDataset

In [237]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [238]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [239]:
print('train[img_path].values :', train['img_path'].values)
print('train[label].values :', train['label'].values)
# print('train_transform :', train_transform)

train_dataset = CustomDataset(
    train['img_path'].values, train['label'].values, train_transform)

train_loader = DataLoader(
    train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values,
                            val['label'].values, test_transform)

val_loader = DataLoader(
    val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


train[img_path].values : ['..\\..\\open\\train\\훼손\\878.png' '..\\..\\open\\train\\훼손\\1030.png'
 '..\\..\\open\\train\\훼손\\124.png' ... '..\\..\\open\\train\\면불량\\82.png'
 '..\\..\\open\\train\\훼손\\378.png' '..\\..\\open\\train\\몰딩수정\\4.png']
train[label].values : [18 18 18 ...  6 18  7]


## Model Define

In [240]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [241]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [242]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

## Run!!

In [243]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/76 [00:00<?, ?it/s]


TypeError: image must be numpy array type

## Inference

In [ ]:
# 테스트 csv 파일 경로 수정
test = pd.read_csv('../../open/test.csv')

In [ ]:
# ======== 테스트 이미지 파일 경로 수정 ========

test_img_paths = []

for i in test['img_path'].values:

    test_img_path = '../../open/' + i.replace('./', '')

    test_img_paths.append(test_img_path)

print('test_img_paths :', test_img_paths)

test_dataset = CustomDataset(test_img_paths, None, test_transform)

# =============================================

test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

## Submission

In [ ]:
# 제출용 추론 결과 csv 파일 경로 수정
submit = pd.read_csv('../../open/sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)